#### Import libraries and packages

In [1]:
import psycopg2
import psycopg2.extras as extras
import pandas as pd
import numpy as np

#### Set connection with postgres database

In [2]:
host = 'postgresfib.fib.upc.edu'
dbname = 'ADSDBjordi.cluet'
user = 'jordi.cluet'
pwd = 'DB151199'
port = 6433
sslmode = 'require'

conn = psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, user, pwd))
cursor = conn.cursor()

---

## Load housing table into formatted zone

##### Read dataframe from CSV file

In [3]:
df = pd.read_csv('data/zenodo_fotocasa_2020_21-12-06_formatted.csv')
df.columns

Index(['ID', 'address', 'bathrooms', 'building_subtype', 'building_type',
       'conservation_state', 'extraction_date', 'discount', 'floor_elevator',
       'is_new_construction', 'link', 'price', 'real_estate', 'real_estate_id',
       'rooms', 'sq_meters', 'neighbourhood', 'neighbourhood_mean_price'],
      dtype='object')

##### Create formatted_zone schema if it does not exist

In [4]:
create_formatted_zone = """CREATE SCHEMA IF NOT EXISTS formatted_zone;"""
cursor.execute(create_formatted_zone)
conn.commit()

##### Create new table in PostgreSQL database

In [5]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.zenodo_fotocasa_2020_21_12_06 (
    ID INTEGER PRIMARY KEY,
    ADDRESS VARCHAR(80),
    BATHROOMS INTEGER,
    BUILDING_SUBTYPE VARCHAR(30),
    BUILDING_TYPE VARCHAR(4),
    CONSERVATION_STATE INTEGER,
    EXTRACTION_DATE DATE,
    DISCOUNT INTEGER,
    FLOOR_ELEVATOR INTEGER,
    IS_NEW_CONSTRUCTION BOOLEAN,
    LINK VARCHAR(255),
    PRICE FLOAT,
    REAL_ESTATE VARCHAR(55),
    REAL_ESTATE_ID VARCHAR(15),
    ROOMS INTEGER,
    SQ_METERS FLOAT,
    NEIGHBOURHOOD VARCHAR(45),
    NEIGHBOURHOOD_MEAN_PRICE FLOAT
);"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [6]:
def execute_values(conn, df, table):
  
    tuples = [tuple(x) for x in df.to_numpy()]
  
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("The dataframe was correctly inserted")
    cursor.close()

In [7]:
execute_values(conn, df, 'formatted_zone.zenodo_fotocasa_2020_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [8]:
sql = "SELECT * from formatted_zone.zenodo_fotocasa_2020_21_12_06;"
pd.read_sql_query(sql, conn).head()

,id,address,bathrooms,building_subtype,building_type,conservation_state,extraction_date,discount,floor_elevator,is_new_construction,link,price,real_estate,real_estate_id,rooms,sq_meters,neighbourhood,neighbourhood_mean_price
0,0,"Carrer Roger de Lluria, Dreta de l'Eixample",3,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/calefa...,5000.0,SOMOS REAL ESTATE SERVICES,9202765680731,3,208.0,la dreta de l'eixample,1307.616842
1,1,Sant Antoni,1,Flat,Flat,1,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/aire-a...,1250.0,TECNOCASA RIERA ALTA MAR,9202751363773,3,93.0,sant antoni,994.909158
2,2,"Carrer Valencia, La Nova Esquerra de l'Eixample",2,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/calefa...,1600.0,SOMOS REAL ESTATE SERVICES,9202765680731,4,129.0,la nova esquerra de l'eixample,1053.832393
3,3,"Carrer Muntaner, Sant Gervasi- Galvany",2,Flat,Flat,0,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/aire-a...,3200.0,SOMOS REAL ESTATE SERVICES,9202765680731,2,133.0,sant gervasi - galvany,1397.273005
4,4,"Rambla del Raval, El Raval",1,Flat,Flat,2,2020-10-28,0,1,False,/es/alquiler/vivienda/barcelona-capital/ascens...,1000.0,ESTUDI JOAN MIRO.SL.,9202762439813,3,67.0,el raval,850.112343


---

## Load barris-districtes table into formatted zone

##### Read dataframe from CSV file

In [9]:
df = pd.read_csv('data/ajunt_barris_2017_21-12-06.csv')
df.columns

Index(['CODI_DISTRICTE', 'NOM_DISTRICTE', 'CODI_BARRI', 'NOM_BARRI'], dtype='object')

##### Create new table in PostgreSQL database

In [10]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.AJUNT_BARRIS_2017_21_12_06 (
    CODI_DISTRICTE INTEGER,
    NOM_DISTRICTE VARCHAR(50),
    CODI_BARRI INTEGER,
    NOM_BARRI VARCHAR(50));"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [11]:
execute_values(conn, df, 'formatted_zone.AJUNT_BARRIS_2017_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [12]:
sql = "SELECT * from formatted_zone.AJUNT_BARRIS_2017_21_12_06;"
pd.read_sql_query(sql, conn).head()

,codi_districte,nom_districte,codi_barri,nom_barri
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc


---

## Load barris-districtes table into trusted zone
As no quality changes are needed.

##### Create trusted_zone schema if it does not exist

In [13]:
create_trusted_zone = """CREATE SCHEMA IF NOT EXISTS trusted_zone;"""
cursor.execute(create_trusted_zone)
conn.commit()

##### Create new table in PostgreSQL database

In [14]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS trusted_zone.AJUNT_BARRIS_2017_21_12_06 (
    CODI_DISTRICTE INTEGER,
    NOM_DISTRICTE VARCHAR(50),
    CODI_BARRI INTEGER,
    NOM_BARRI VARCHAR(50));"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [15]:
execute_values(conn, df, 'trusted_zone.AJUNT_BARRIS_2017_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [16]:
sql = "SELECT * from trusted_zone.AJUNT_BARRIS_2017_21_12_06;"
pd.read_sql_query(sql, conn).head()

,codi_districte,nom_districte,codi_barri,nom_barri
0,1,Ciutat Vella,1,el Raval
1,1,Ciutat Vella,2,el Barri Gòtic
2,1,Ciutat Vella,3,la Barceloneta
3,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera"
4,2,Eixample,5,el Fort Pienc


---

## Load crime table into formatted zone

##### Read dataframe from CSV file

In [17]:
df = pd.read_excel('data/ajunt_crime_2020_21-12-06_final.xlsx')
print(len(df.columns))
df.columns

25


Index(['Disctricte', 'Furt', 'Estafes', 'Danys',
       'Robatori amb violència i/o intimidació',
       'Robatori amb força interior vehicle', 'Robatori amb força', 'Lesions',
       'Apropiació indeguda', 'Amenaces', 'Robatori i furt d'us de vehicle',
       'Ocupació immobles', 'Contra la salut pública', 'Abusos sexuals',
       'Entrada a domicili aliè', 'Agressions sexuals', 'Convivència veïnal',
       'Vigilància policial', 'Activitats molestes en espais públics',
       'Actes contra la propietat privada', 'Incendis',
       'Estupefaents/psicotròpics', 'Agressions',
       'Nombre de proves d'alcoholèmia', 'Nombre de drogo tests'],
      dtype='object')

##### Rename columns

In [18]:
df.columns = ['Districte', 'Furt', 'Estafes', 'Danys', 'Rob_viol_intim', 'Rob_en_vehicle', 'Rob_força', 'Lesions', 'Aprop_indeg', 'Amenaces', 'Rob_de_vehicle', 'Ocupacions', 'Salut_pub', 'Abusos_sex', 'Entrada_domicili', 'Agressio_sex', 'Conviv_veinal', 'Vigilancia_poli', 'Molesties_espai_pub', 'Contra_prop_priv', 'Incendis', 'Estupefaents', 'Agressions', 'Proves_alcohol','Proves_droga']

##### Create formatted_zone schema if it does not exist

In [19]:
create_formatted_zone = """CREATE SCHEMA IF NOT EXISTS formatted_zone;"""
cursor.execute(create_formatted_zone)
conn.commit()

##### Create new table in PostgreSQL database

In [20]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.ajunt_crime_2020_21_12_06 (
    DISTRICTE VARCHAR(50),
    FURT INTEGER,
    ESTAFES INTEGER,
    DANYS INTEGER,
    ROB_VIOL_INTIM INTEGER,
    ROB_EN_VEHICLE INTEGER,
    ROB_FORÇA INTEGER,
    LESIONS INTEGER,
    APROP_INDEG INTEGER,
    AMENACES INTEGER,
    ROB_DE_VEHICLE INTEGER,
    OCUPACIONS INTEGER,
    SALUT_PUB INTEGER,
    ABUSOS_SEX INTEGER,
    ENTRADA_DOMICILI INTEGER,
    AGRESSIO_SEX INTEGER,
    CONVIV_VEINAL INTEGER,
    VIGILANCIA_POLI INTEGER,
    MOLESTIES_ESPAI_PUB INTEGER,
    CONTRA_PROP_PRIV INTEGER,
    INCENDIS INTEGER,
    ESTUPEFAENTS INTEGER,
    AGRESSIONS INTEGER,
    PROVES_ALCOHOL INTEGER,
    PROVES_DROGA INTEGER
);"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [21]:
execute_values(conn, df, 'formatted_zone.ajunt_crime_2020_21_12_06')

The dataframe was correctly inserted


##### Check correct insertion

In [22]:
sql = "SELECT * from formatted_zone.ajunt_crime_2020_21_12_06;"
pd.read_sql_query(sql, conn).head()

,districte,furt,estafes,danys,rob_viol_intim,rob_en_vehicle,rob_força,lesions,aprop_indeg,amenaces,...,agressio_sex,conviv_veinal,vigilancia_poli,molesties_espai_pub,contra_prop_priv,incendis,estupefaents,agressions,proves_alcohol,proves_droga
0,Ciutat Vella,11827,1209,1110,2174,915,866,754,1493,415,...,43,7645,10703,6353,4089,444,1117,601,3520,278
1,L'Eixample,14157,3390,1714,1761,1484,1543,722,833,401,...,32,11474,4721,4445,2178,793,451,477,3770,326
2,Sants-Montjuïc,5605,2024,1268,1050,1075,958,689,401,456,...,16,6783,2933,4477,1250,458,265,353,4286,738
3,Les Corts,2489,999,478,379,457,367,132,118,101,...,3,2030,1405,1394,369,150,55,93,1379,65
4,Sarrià-Sant Gervasi,2441,1822,792,590,589,757,191,137,120,...,11,4842,2641,2318,462,302,139,151,4195,291


---

## Load district population and surface table into formatted zone

##### Read dataframe from CSV file

In [23]:
df = pd.read_excel('data/ajunt_districtes_2021_21-12-24.xlsx')
print(len(df.columns))
df.columns

3


Index(['Districte', 'Superfície', 'Població'], dtype='object')

##### Rename columns

In [24]:
df.columns = ['Districte', 'Superficie', 'Poblacio']

##### Create new table in PostgreSQL database

In [25]:
sqlCreateTable = """CREATE TABLE IF NOT EXISTS formatted_zone.ajunt_districtes_2021_21_12_24 (
    DISTRICTE VARCHAR(50),
    SUPERFICIE FLOAT,
    POBLACIO INTEGER
);"""
cursor.execute(sqlCreateTable)
conn.commit()

##### Insert rows into table

In [26]:
execute_values(conn, df, 'formatted_zone.ajunt_districtes_2021_21_12_24')

The dataframe was correctly inserted


##### Check correct insertion

In [27]:
sql = "SELECT * from formatted_zone.ajunt_districtes_2021_21_12_24;"
pd.read_sql_query(sql, conn)

,districte,superficie,poblacio
0,Ciutat Vella,420.5,107858
1,Eixample,746.4,270331
2,Sants-Montjuïc,2288.0,187425
3,Les Corts,601.1,82532
4,Sarrià-Sant Gervasi,1991.6,150888
5,Gràcia,422.4,123614
6,Horta-Guinardó,1192.0,174447
7,Nou Barris,805.6,173864
8,Sant Andreu,659.2,151960
9,Sant Martí,1043.7,241263
